# Homeworks - Big Data and Public Policy Class

*Name*: Andreas Eckmann

*E-Mail*: eckmanna@ethz.ch

In [2]:
from IPython.display import display
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint

import tweepy
from tweepy import OAuthHandler

import yfinance as yf
import re

**Step 1a: `X` variables** [week 2]
- Fetch the data using the **twitter API** or any other API or website that you are interested in.
- Beware of the rate limits and organize your program so as to overcome them if needed
- The data should include some text, but might also have other interesting variables (retweets, favorites...)
- Create some (non-text based) `X_num` variables that you will use for the prediction

In [15]:
#this you get when you make create an application on twitter as a dev
consumer_key="7X8q1LteL1qOLRg4DSoiI0lyk"
consumer_secret="Bb08vV5XoxDEP4SGLJfkvEpuwvxEDvzVSRGTptzQCjC9XVmquP"
access_token="1230145588659998721-EQb8DoMYnorBtqAuEPMZtZcXumzuAe"
access_token_secret="mzurpOu1LWhj0NU9pctgYZ6OBzhQvyqw8hvRNpe3yI4Qc"

## Accessing the Twitter API using `tweepy`

Twitter requires all requests to use `OAuth` for authentication

In [17]:
# Authenticate to Twitter
auth = OAuthHandler(consumer_key, consumer_secret) #creating an OAuthHandler instance
auth.set_access_token(access_token, access_token_secret)

# We specify wait_on_rate_limit_notify==True & wait_on_rate_limit=True. The API method will wait once you’ve reached your rate limit and prints out a message
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [23]:
# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")


Authentication OK


In [ ]:
# most recent tweets about Beyond Meat
tweets = api.search(q="Beyond Meat", lang="en") # TODO: How is it set to recent tweets?
for tweet in tweets:
    print(tweet.text) # printing the first tweet

In [18]:
target='BeyondMeat' #'BeyondMeat'
user = api.get_user(target) # argument = id, user_id, screen_name
pprint(user._json)


NameError: name 'api' is not defined

In [38]:
# Some attributes of the `user` object:
print("Name:", user.name)
print("Screen name:", user.screen_name)
print("Number of followers:" ,  user.followers_count)
pprint("description: " + user.description)
pprint("Number of tweets published: " + str(user.statuses_count))
pprint("friends_count: " + str(user.friends_count))

Name: Beyond Meat
Screen name: BeyondMeat
Number of followers: 108771
" description: Meat that's made from plants, made for meat lovers. \n#GoBeyond"
' Number of tweets published: 24010'
' friends_count: 692'


In [21]:
def fetch_tweets(account, n_tweets):
    """Fetch data from Beyond Meat twitter account"""
    df_out = pd.DataFrame()
    
    try:   
        # Put the tweets into a data frame object
        tweet_count = 0
        for tweet in tweepy.Cursor(api.user_timeline, 
                                   screen_name=target, 
                                   tweet_mode ='extended',
                                   include_rts = False).items(n_tweets):
    
                    # 1. Transform the json into a dataframe
                    df_tweet = pd.DataFrame.from_dict(tweet._json, orient='index', 
                                                      columns=[tweet_count]) # , sleep_on_rate_limit=True
    
                    # 2. adds screen name as a row
                    #df_tweet=df_tweet.append(pd.DataFrame({tweet_count:[target]}, index=['twitter_handle']))
    
                    # 3. Concatenate the tweet dataframe to the df_tweets dataframe
                    df_out = pd.concat([df_tweet, df_out], axis=1)
    
                    # count the number of tweets fetched
                    tweet_count += 1 
    
        time.sleep(1)
    
    # except if TweepError arises
    except tweepy.TweepError: #the error arises when the user has protected tweets
        print("Failed to run the command "
              "on user {}, Skipping...".format(account))
    
    # except if RateLimitError arises
    except tweepy.RateLimitError:
        print("resource usage limit: {} skipped".format(account))
        time.sleep(15 * 60) # wait 15 mins
        
    return df_out


**Step 1b: continuous `y` variables** [week 2]
- Fetching the data: 
    - if you work on the suggested idea, you can easily access daily stock prices using the [`yfinance` package](https://pypi.org/project/yfinance/)  (see below)
    - otherwise, you can find some interesting data listed in the syllabus

In [5]:
bynd = yf.Ticker("BYND")
# get stock info
bynd.info

{'zip': '90245',
 'sector': 'Consumer Defensive',
 'fullTimeEmployees': 472,
 'longBusinessSummary': 'Beyond Meat, Inc., a food company, engages in the provision of revolutionary plant-based meats in the United States and internationally. It offers its products in plant-based platforms of beef, pork, and poultry in ready-to-cook and ready-to-heat formats. The company sells its products through grocery, mass merchandiser, club and convenience store, and natural retailer channels, as well as direct to consumer and various food-away-from-home channels. The company was formerly known as Savage River, Inc. and changed its name to Beyond Meat, Inc. in September 2018. Beyond Meat, Inc. was founded in 2009 and is headquartered in El Segundo, California.',
 'city': 'El Segundo',
 'phone': '866 756 4112',
 'state': 'CA',
 'country': 'United States',
 'companyOfficers': [],
 'website': 'http://www.beyondmeat.com',
 'maxAge': 1,
 'address1': '119 Standard Street',
 'industry': 'Packaged Foods',
 '

In [10]:
# get historical market data
hist = bynd.history(period="max")
pprint(hist)

              Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                         
2019-05-02   46.00   72.95   45.00   65.75  23119000          0             0
2019-05-03   72.00   74.00   65.66   66.79  13139400          0             0
2019-05-06   62.73   74.84   62.50   74.79   8746200          0             0
2019-05-07   77.14   85.45   75.00   79.17  16532100          0             0
2019-05-08   83.61   85.38   70.79   72.25  14696300          0             0
2019-05-09   70.50   73.20   67.10   68.27   6284500          0             0
2019-05-10   69.09   69.33   61.60   66.22   4888000          0             0
2019-05-13   65.46   71.96   63.36   69.50   4773400          0             0
2019-05-14   72.48   80.75   71.12   79.68   7079600          0             0
2019-05-15   79.00   93.00   74.55   86.92  18356200          0             0
2019-05-16   90.10   96.79   89.51   92.92  13900300          0 

In [7]:
# show actions (dividends, splits)
bynd.actions

,Dividends,Stock Splits
Date,,


In [13]:
# show earnings
bynd.earnings
bynd.quarterly_earnings

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


**Step 1c: merge `X_num` and `y`** [week 2]
- Beware of the temporality: in the case of the proposed study on stock market prices, you will have to deal with the fact that the X is at the tweet level while `y` is daily. 


***Milestone 1 - March 24th*** *You can submit the previous steps as a first notebook.*

**Sample split.** Do the standard 80% / 20% training/test split using all days in the data. In addition, do a separate temporal split where the training set is the first 80% of days in the time series.

**For all machine learning models**, report performance measure in test and train samples.

**Step 2a: estimate different regression models using `X_num` and `y`** [week 3]

***Milestone 2 - March 31th***  *You can submit the previous steps as a second notebook.*

**Step 3: text analysis** [week 4]
- Featurize tweets (or another text dataset related to your subject): transform the text into a standard document-level dataset `X_doc`

**Step 2b: estimate different regression models using `X_doc` and `y`** [week 3]

**Step 4: estimate a classification models** [week 5]
- propose a categorical variable `y_calc` that you can compute from the continuous one (`y`) (e.g. positive or negative growthin stock prices). For the `X` dimension, you can use `X_doc` or `X_num` or both. 
- you can use any other categorical variable that you find relevant 

***Milestone 3 - April 21th***  *You can submit the previous steps as a third notebook.*

**Step 6: Dimension reduction** [week 6]
- Use one of the dimension reductions methods to dimension-reduce the features
    - PCA or topic model (LDA or STM) or k-means clustering on the featurized text `X_doc`
- Run another classifier

**Step bonus: Econometric identification** [week 8]
- Find an exogenous shock affecting this firm (but not all the firms) and a control group of firms not affected
    - example: a natural disaster/shock to the exchange rate/change in ownership... affects the functionning of this firm but not the other firms of the stock market index.  
- scale up the previous data collection to the firms in the control group
- use one of the technique studied in class to causaly identify the impact of the exogenous shock on the stock market of the affected firm

***Milestone 4 - May 17th*** *You can submit the previous steps as a last notebook.*

---------------------------
**Requirement for completion grade**: 
- the homework should be done in **groups of 1 or 2 persons** 
- the homework will have to be submitted as a (Python or R) **notebooks**
- you can give them back at the indicated **milestone** or on May 17th (no homework will be accepted after May 17th)
- the notebooks should contain at least **three graphs** (overall)
- each notebook should run from the beginning to then end, but:
    - beware of paths to folders that are on your computer but not on mine
    - if you have such a path, you can: 
        - have the data downloaded in the notebook directly
        - import your data in a github (or any other online storage) folder that the notebook can access  
    - indicate if the notebook takes more than 30 minutes to run
- do not write us emails regarding the homework, ask your questions on the **forum**, it will benefit everyone for sure! 

---------------------------